In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from risk_measure import *

In [2]:
# The quantile function is simply the inverse CDF of a distribution
%psource RiskMeasure.Parametric.CDF

        def CDF(self, x):
            return self._distribution.cdf(x)


In [3]:
# Value at Risk
%psource RiskMeasure.Parametric.valueAtRisk

        def valueAtRisk(self, alpha=0.05, **kwargs):
            distribution = self._distribution
            valueAtRisk = distribution.ppf(q=alpha,**kwargs)
            return valueAtRisk


In [4]:
# Expected Shortfall
%psource RiskMeasure.Parametric.expectedShortfall

        def expectedShortfall(self, alpha=0.05, **kwargs):
            distribution = self._distribution
            valueAtRisk = self.valueAtRisk(alpha, **kwargs)

            if isinstance(distribution, stats.rv_discrete):
                firstTerm = distribution.expect(lambda x: -x, ub=valueAtRisk, conditional=True, args=(kwargs.values()) ) # mind the order of value in "**kwargs"
            else: firstTerm = distribution.expect(lambda x: -x, ub=valueAtRisk, conditional=True, **kwargs ) # Thanks to an amazing library - scipy
            alphaHat = 1 - distribution.cdf(distribution.ppf(1-alpha,**kwargs ),**kwargs )
            secondTerm = -valueAtRisk*( alphaHat - alpha )
            
            expectedShortfall = -(firstTerm + secondTerm)
            return expectedShortfall


In [5]:
print("The value at risk of the standard normal distribution at 95% confidentblevel is", RiskMeasure.Parametric(stats.norm).valueAtRisk(alpha=0.05))
print("The expected shortfall of the standard normal distribution at 95% confidentblevel is", RiskMeasure.Parametric(stats.norm).expectedShortfall(alpha=0.05))

The value at risk of the standard normal distribution at 95% confidentblevel is -1.6448536269514729
The expected shortfall of the standard normal distribution at 95% confidentblevel is -2.0627128078189143


## Let's compare

The parametric method and the non-parametric method
- First, simulate random numbers with a specific distribution
- use the non-parametric method with the numbers and compare with the parametric method

### Exponential Distribution

In [6]:
# random numbers
np.random.seed(42)
randExpo = np.random.exponential(scale=2, size=1000000)

VaR_expo_sim = RiskMeasure.NonParametric(randExpo).valueAtRisk(alpha=0.1)
ES_expo_sim = RiskMeasure.NonParametric(randExpo).expectedShortfall(alpha=0.1)
print("The value at risk by the simulation method is ", VaR_expo_sim)
print("The expected shortfall by the simulation method is ", ES_expo_sim)

VaR_expo_para = RiskMeasure.Parametric(stats.expon).valueAtRisk(scale=2, alpha=0.1)
ES_expo_para = RiskMeasure.Parametric(stats.expon).expectedShortfall(scale=2, alpha=0.1)
print("The value at risk by the parametric method is ", VaR_expo_para)
print("The expected shortfall by the parametric method is ", ES_expo_para)

The value at risk by the simulation method is  0.21109151861322278
The expected shortfall by the simulation method is  0.10405204792909421
The value at risk by the parametric method is  0.21072103131565262
The expected shortfall by the parametric method is  0.1035107181591266


### Normal Distribution

In [ ]:
randNorm = np.random.normal(loc=5,scale=1, size=1000000)

VaR_norm_sim = RiskMeasure.NonParametric(randNorm).valueAtRisk(alpha=0.1)
ES_norm_sim = RiskMeasure.NonParametric(randNorm).valueAtRisk(alpha=0.1)
print("The value at risk by the simulation method is ", VaR_norm_sim)
print("The expected shortfall by the simulation method is ", ES_norm_sim)

VaR_norm_para = RiskMeasure.Parametric(stats.norm).valueAtRisk(loc=5, scale=1, alpha=0.1)
ES_norm_para = RiskMeasure.Parametric(stats.norm).expectedShortfall(loc=5, scale=1, alpha=0.1)
print("The value at risk by the parametric method is ", VaR_norm_para)
print("The expected shortfall by the parametric method is ", ES_norm_para)

The value at risk by the simulation method is  3.714523869376157
The expected shortfall by the simulation method is  3.714523869376157
The value at risk by the parametric method is  3.7184484344553996
The expected shortfall by the parametric method is  3.245016680675133


### Binomial Distribution

In [8]:
randBin = np.random.binomial(n=300,p=0.32,size=1000000)

VAR_bin_sim = RiskMeasure.NonParametric(randBin).valueAtRisk(alpha=0.1)
ES_bin_sim = RiskMeasure.NonParametric(randBin).expectedShortfall(alpha=0.1)
print("The value at risk by the simulation method is ", VAR_bin_sim)
print("The expected shortfall by the simulation method is ", ES_bin_sim)

VAR_bin_para = RiskMeasure.Parametric(stats.binom).valueAtRisk(n=300, p=0.32, alpha=0.1)
ES_bin_para = RiskMeasure.Parametric(stats.binom).expectedShortfall(n=300, p=0.32, alpha=0.1)
print("The value at risk by the parametric method is ", VAR_bin_para)
print("The expected shortfall by the parametric method is ", ES_bin_para)

The value at risk by the simulation method is  86.0
The expected shortfall by the simulation method is  81.98462
The value at risk by the parametric method is  86.0
The expected shortfall by the parametric method is  82.4170731911514
